In [4]:
cd

/Users/keith_tetrad


In [5]:
import __main__
import pandas as pd

__main__.pymol_argv = ['pymol', '-qc']
import pymol
from pymol import cmd, stored

pymol.finish_launching()

In [6]:
# cmd.set('dot_solvent', 1)
# cmd.set('dot_density', 3)

# cmd.load('kw_dms/6xm4.pdb')  # use the name of your pdb file
# stored.residues = []
# cmd.iterate('name ca', 'stored.residues.append((resi, resn))')

# # Calculate SASA for each residue
# sasa_per_residue = []
# for resi, resn in stored.residues:
#     area = cmd.get_area('resi %s' % resi)
#     sasa_per_residue.append((resi, resn, area))

# # Create a DataFrame
# df = pd.DataFrame(sasa_per_residue, columns=['Residue Number', 'Residue Identity', 'SASA'])

    
# print(df)
# print("Total SASA:", cmd.get_area('all'))  # To check the total area

     Residue Number Residue Identity        SASA
0                27              ALA  252.676834
1                28              TYR  472.798737
2                29              THR   88.997017
3                30              ASN  139.515320
4                31              SER    4.915101
...             ...              ...         ...
3139           1143              PRO  220.134689
3140           1144              GLU  347.476288
3141           1145              LEU  197.068588
3142           1146              ASP  261.356964
3143           1147              SER  247.679779

[3144 rows x 3 columns]
Total SASA: 219937.078125


In [7]:
#reference SASA based on Tien et al. 2013 theoretical
reference_sasa = {
    'ALA': 129,
    'ARG': 274,
    'ASN': 195,
    'ASP': 193,
    'CYS': 167,
    'GLU': 223,
    'GLN': 225,
    'GLY': 104,
    'HIS': 224,
    'ILE': 197,
    'LEU': 201,
    'LYS': 236,
    'MET': 224,
    'PHE': 240,
    'PRO': 159,
    'SER': 155,
    'THR': 172,
    'TRP': 285,
    'TYR': 263,
    'VAL': 174
}

In [8]:
# Function to calculate rSASA
def calculate_rSASA(row):
    ref_sasa = reference_sasa.get(row['Residue Identity'])
    if ref_sasa:
        return (row['SASA'] / ref_sasa) * 100
    else:
        return None

In [12]:
# Apply the function to each row
relative_SASA = df.copy()

# Apply the function to each row
df['rSASA'] = df.apply(calculate_rSASA, axis=1)

In [19]:
def determine_accessibility(row):
    if row['rSASA'] > 25:
        return "Exposed"
    else:
        return "Burried"

In [20]:
df['surface_accessibility'] = df.apply(determine_accessibility, axis=1)
print(df)

     Residue Number Residue Identity        SASA       rSASA  \
0                27              ALA  252.676834  195.873515   
1                28              TYR  472.798737  179.771383   
2                29              THR   88.997017   51.742452   
3                30              ASN  139.515320   71.546318   
4                31              SER    4.915101    3.171033   
...             ...              ...         ...         ...   
3139           1143              PRO  220.134689  138.449490   
3140           1144              GLU  347.476288  155.818963   
3141           1145              LEU  197.068588   98.044074   
3142           1146              ASP  261.356964  135.418116   
3143           1147              SER  247.679779  159.793406   

     surface_accessibility  
0                  Exposed  
1                  Exposed  
2                  Exposed  
3                  Exposed  
4                  Burried  
...                    ...  
3139               Exposed  

In [21]:
#write to a csv
df.to_csv('kw_dms/sasa_data.csv', index=False)